# Module 8 Final Project - Chatham

In [4]:
import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import xarray as xr
from sklearn.preprocessing import StandardScaler
import pandas as pd
from eofs.xarray import Eof

Create a dataset that contains the monthly means of Sea Surface Temperature anomalies from Jan 2003-Dec 2020 over the intended region (60°N to 60°S, 0-360) masked out over land. The data and land sea mask is available here: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download

In [6]:
dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "year": [
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "variable": [
        "land_sea_mask",
        "sea_surface_temperature"
    ],
    "area": [60, 0, -60, 360]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

Exception: Missing/incomplete configuration file: /home/alison8/.cdsapirc